In [100]:
import random
from matplotlib import pyplot as plt
alphabet = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ,.! "
target = "Hello World!"

class Individual:
     def __init__(self, string, fitness=0):
        self.string = str(string)
        self.fitness = fitness
        
from difflib import SequenceMatcher

def choices(string, k):
    s = ''
    for i in range(5):
        s = s+str(random.choice(alphabet))
    return s

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def spawn_population(length=26, size=100): 
    pop = []
    for i in range(size):
        #string = ''.join(random.choices(alphabet,k=length))
        string = ''.join(choices(alphabet,k=length))
        individual = Individual(string)
        pop.append(individual)
        return pop

In [101]:
def recombine(p1_, p2_): 
    p1 = p1_.string
    p2 = p2_.string
    child1 = []
    child2 = []
    cross_pt = random.randint(0,len(p1))
    child1.extend(p1[0:cross_pt])
    child1.extend(p2[cross_pt:])
    child2.extend(p2[0:cross_pt])
    child2.extend(p1[cross_pt:])
    c1 = Individual(''.join(child1))
    c2 = Individual(''.join(child2))
    return c1, c2

In [102]:
def mutate(x, mut_rate=0.01): #B
    new_x_ = []
    for char in x.string:
        if random.random() < mut_rate:
            # new_x_.extend(random.choices(alphabet,k=1))
            new_x_.extend(choices(alphabet,k=1))
        else:
            new_x_.append(char)
    new_x = Individual(''.join(new_x_))
    return new_x

In [103]:
def evaluate_population(pop, target): 
    avg_fit = 0
    for i in range(len(pop)):
        fit = similar(str(pop[i]), target)
        pop[i].fitness = fit
        avg_fit += fit
    avg_fit /= len(pop)
    return pop, avg_fit

In [104]:
def next_generation(pop, size=100, length=26, mut_rate=0.01): 
    new_pop = []
    while len(new_pop) < size:
        # parents = random.choices(pop,k=2, weights=[x.fitness for x in pop])
        parents = choices(pop,k=2)
        offspring_ = recombine(parents[0],parents[1])
        child1 = mutate(offspring_[0], mut_rate=mut_rate)
        child2 = mutate(offspring_[1], mut_rate=mut_rate),
        offspring = [child1, child2]
        new_pop.extend(offspring)
    return new_pop

In [105]:
num_generations = 100
population_size = 3000
str_len = len(target)
mutation_rate = 0.001
pop_fit = []
pop = spawn_population(size=population_size, length=str_len)
for gen in range(num_generations):
    pop, avg_fit = evaluate_population(pop, target)
    pop_fit.append(avg_fit) 
    new_pop = next_generation(pop, size=population_size, length=str_len, mut_rate=mutation_rate)
    pop = new_pop

AttributeError: 'str' object has no attribute 'string'

In [106]:
import torch
from torch.nn import functional as F

class SimulatorState(torch.nn.Module):
    def __init__(self):
        super(SimulatorState, self).__init__()
        self.conv1 = torch.nn.Conv2d(4, 8, kernel_size=3, padding=1)
        self.conv2 = torch.nn.Conv2d(8, 16, kernel_size=3, padding=1)
        self.conv3 = torch.nn.Conv2d(16, 32, kernel_size=3, padding=1)

        self.s_fc1 = torch.nn.Linear(512, 99)

        self.action_fc1 = torch.nn.Linear(4, 1)

        self.fc1 = torch.nn.Linear(100, 50)
        self.fc2 = torch.nn.Linear(50, 16)

        #         self.s_fc1 = torch.nn.Linear(50, 16)

        self.reward_fc1 = torch.nn.Linear(50, 30)
        self.reward_fc2 = torch.nn.Linear(30, 20)
        self.reward_fc3 = torch.nn.Linear(20, 3)

    def forward(self, x):
        state = x[:, :64]
        a_x = x[:, 64:]

        num_batch = state[0]

        s_x = state.reshape(-1, 4, 4, 4)

        s_x = F.relu(self.conv1(s_x))
        s_x = F.relu(self.conv2(s_x))
        s_x = F.relu(self.conv3(s_x))
        s_x = s_x.view(-1, 8 * 64)
        s_x = self.s_fc1(s_x)

        a_x = self.action_fc1(a_x)

        x = torch.cat((s_x, a_x), dim=1)

        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        #         state_copy = state.reshape(-1,4,16).clone()
        #         state_copy[np.arange(num_batch)][0][:] = 0
        #         state_copy[np.arange(num_batch),s_x] = 1

        #         r_x = F.relu(self.reward_fc1(state_copy))
        #         r_x = F.relu(self.reward_fc2(r_x))
        #         r_x = self.reward_fc3(r_x)

        return F.softmax(x)


class SimulatorReward(torch.nn.Module):
    def __init__(self):
        super(SimulatorReward, self).__init__()
        self.conv1 = torch.nn.Conv2d(4, 8, kernel_size=3, padding=1)
        self.conv2 = torch.nn.Conv2d(8, 16, kernel_size=3, padding=1)
        self.conv3 = torch.nn.Conv2d(16, 32, kernel_size=3, padding=1)

        self.fc1 = torch.nn.Linear(512, 200)
        self.fc2 = torch.nn.Linear(200, 100)
        self.fc3 = torch.nn.Linear(100, 3)

    def forward(self, x):
        x = x.reshape(-1, 4, 4, 4)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.conv3(x)

        x = x.view(-1, 512)

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return F.softmax(x)

In [108]:
import torch
from torch.autograd import Variable
import numpy as np
from torch.nn import functional as F
from matplotlib import pyplot as plt
import sys
sys.path.append("script")
from Gridworld import Gridworld
import random
from tqdm import tqdm
import math
from copy import deepcopy

from argparse import ArgumentParser

from simulator import SimulatorState, SimulatorReward
from buffer import ExperienceReplay

action_set = {
    0: 'u',
    1: 'd',
    2: 'l',
    3: 'r',
}


def running_mean(x, N=500):
    cumsum = np.cumsum(np.insert(x, 0, 0))
    return (cumsum[N:] - cumsum[:-N]) / float(N)


def encode_game_progress(reward):
    if reward == 0:
        return 0  # in progress
    elif reward == 1:
        return 1  # won
    else:
        return 2  # lost


def decode_game_progress(index):
    if index == 0:
        return 0  # in progress
    elif index == 1:
        return 1  # won
    else:
        return -1  # lost


def convert_to_state(state):
    s_ = state.reshape(1, 4, 16)
    s = s_.max(dim=2)
    output = torch.zeroes(1, 4, 16)
    output[0][s[0][0]] = 1
    output[0][s[0][1]] = 1
    output[0][s[0][2]] = 1
    output[0][s[0][3]] = 1
    return output


ImportError: No module named tqdm

不足モジュール
- tqdm


In [110]:
epochs=5000
mode='rand'
warm_up_period=1000
batch_size=1000
max_steps=15
lr = 0.0015
device = torch.device('cpu')
simulator_s = SimulatorState().to(device)
simulator_r = SimulatorReward().to(device)
opt_s = torch.optim.Adam(simulator_s.parameters(), lr=lr)
opt_r = torch.optim.Adam(simulator_r.parameters(), lr=lr)

loss_fn_state = torch.nn.CrossEntropyLoss()
loss_fn_reward = torch.nn.CrossEntropyLoss(weight=torch.Tensor([1, 50, 50]))
losses = []
buffer = ExperienceReplay()

progress = tqdm(range(epochs))
for epoch_num in progress:
    game = Gridworld(mode=mode)
    z = 0
    for step_num in args.max_steps:
        # get starting state
        state = torch.from_numpy(game.board.render_np()).float().reshape(64, )
        # take random action
        action_ = np.random.randint(0, 4)
        action = action_set[action_]
        action_vec = torch.zeros(4, )
        action_vec[action_] = 1

        game.makeMove(action)
        next_state = torch.from_numpy(game.board.render_np()).float()
        reward_ = encode_game_progress(game.reward())
        buffer.add([(state, action_vec, next_state[0].argmax(), reward_, next_state)])

        if len(buffer) >= args.warm_up_period:
            minibatch = buffer.sample(args.batch_size)
            opt_s.zero_grad()
            opt_r.zero_grad()
            states, actions, next_states_i, rewards, next_states = zip(*minibatch)
            states = torch.stack(states).to(device)
            actions = torch.stack(actions).to(device)
            pred_states, _ = simulator_s(torch.cat((states, actions), dim=1)).to(device)
            next_states_i = torch.stack(next_states_i).to(device)
            pred_rewards = simulator_r(torch.stack(next_states)).to(device)
            loss_state = loss_fn_state(pred_states, next_states_i)
            loss_reward = loss_fn_reward(pred_rewards, torch.Tensor(rewards).long())
            overall_loss = loss_state + loss_reward * 0.5
            overall_loss.backward()
            opt_r.step()
            opt_s.step()
            progress.set_description(
                "{:10.3f} {:10.3f}".format(loss_state.detach().numpy(), loss_reward.detach().numpy()))
            losses.append([loss_state.detach().numpy(), loss_reward.detach().numpy()])
        if game.reward() in [1, -1]:
            break
losses = np.array(losses)

NameError: name 'ExperienceReplay' is not defined